In [1]:
import numpy as np
import pandas as pd
import math
train = pd.read_csv('mnist_train.csv')
test = pd.read_csv('mnist_test.csv')

In [2]:
train.head(2)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_labels = train['label'].values
test_labels = test['label'].values

In [4]:
train.drop('label', axis=1,inplace=True)

In [5]:
test.drop('label', axis=1,inplace=True)

In [6]:
X = train.values
y = test.values

# STOCHASTIC VERSION

In [7]:
from scipy.special import expit
class NN:
    def __init__(self,X=X,y=train_labels,testX=y,testy=test_labels,output_size=10,
                 iterations=10,lr=0.1,size=2500):
        
        #When testing ideal params you're going to want to reduce
        #the size of X from 60k to like 1k just to see what works
        #best can even go smaller maybe 500 for the first couple
        np.random.seed(69)
        #limiting size to 100 for testing
        #change to larger if you want 100 should be plenty
        self.lr = lr
        self.X = X[:size] 
        self.y = y[:size]
        self.testX=testX[:size]
        self.testy=testy[:size]
        self.w1 = np.random.rand(len(self.X[0]),len(self.X[0])) * np.sqrt(1 / len(self.X[0]))
        self.w2 = np.random.rand(len(self.X[0]), output_size)* np.sqrt(1 / output_size)
        self.bias = np.random.rand()
        self.output_size = output_size
        self.bias2 = np.random.rand()
        #gonan try to do activation function to both 
        #hidden and final we can adjust this
        #self.activation = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
        #self.deriv = np.vectorize(lambda x: self.activation(x) * (1 - self.activation(x)))
        self.activation = expit
        self.deriv = lambda x: self.activation(x) * (1 - self.activation(x))
        self.count = 1
        for i in range(iterations):
            self.right_or_wrong = []
            for x, y in zip(self.X,self.y):
                hidden,final = self.forward(x.reshape(1,-1))
                cost = self.back(hidden,final,y,x.reshape(1,-1))
                answer = final.argmax()
                #print(f'answer: {answer} correct value: {y}')
                self.right_or_wrong.append(1) if answer == y else self.right_or_wrong.append(0)
            self.count+=1
            if self.count % 10 == 0:
                print(f'The cost after iteration {self.count}: {cost}')
                print(f'Correctly classified {sum(self.right_or_wrong)/len(self.right_or_wrong) * 100}%')
                print('------------------------------------------------------')
    
    def forward(self,X):
        #in the example you only apply the activation func
        #in the final output i'll try both
        hidden_output = self.activation(X @ self.w1 + self.bias)
        final_output = self.activation(hidden_output @ self.w2 + self.bias2)
        return hidden_output, final_output
        
    def back(self,hidden_output,final_outputs,y,x):
        #y = np.array([0 if i == 0 else 1 for i in range(self.output_size)])
        y = np.array([0 if i!= y else 1 for i in range(self.output_size)])
        results = final_outputs - y
        cost = np.sum((final_outputs - y)**2) / len(results)
        #don't think i need this
        #you only need the difference between outputs
        #so in ours it's just results
        #delta_output = results * self.deriv(results)
        
        #THIS MAY BE ALL KIND OF FUCKED UP
        
        #layer_2_delta is just results
        #layer_1_delta = results @ self.w1.T * self.deriv(hidden_output)
        layer_1_delta = results @ self.w2.T * self.deriv(hidden_output)
        
        #delta_hidden = results @ self.w2.T * self.deriv(hidden_output)
        self.w2 -= self.lr * hidden_output.T @ results
        self.w1 -= self.lr * x.T @ layer_1_delta
        return cost
    
    
    def testing(self):
        #self.testX=testX[:100]
        #self.testy=testy[:100]
        results = []
        for i,y in zip(self.testX,self.testy):
            one = self.activation(i.reshape(1,-1) @ self.w1 + self.bias)
            two = self.activation(one @ self.w2 + self.bias2)
            answer = two.argmax()
            results.append(1) if answer == y else results.append(0)
        print('TESTING RESULTS')
        print('----------------------------------')
        print(f'Percentage of correct classification: {sum(results)/len(results) * 100}')

In [8]:
#testNN = NN(iterations=1000, size=1000)

In [9]:
#testNN.testing()

# BATCHED VERSION

In [45]:
from scipy.special import expit

from scipy.special import expit
class BatchNN:
    def __init__(self,X=X,y=train_labels,testX=y,testy=test_labels,
                 output_size=10,iterations=100,lr=.1, batch_size=10, size=2500):
        
        np.random.seed(69)
        self.lr = lr
        self.X,self.y = self.batchify(X[:size], y[:size],batch_size)
        #self.testX,self.testy = self.batchify(testX[:size], testy[:size],batch_size)
        self.testX = testX[:size]
        self.testy = testy[:size]
        self.w1 = np.random.rand(len(self.X[0][0]),len(self.X[0][0])) * np.sqrt(1 / len(self.X[0][0]))
        self.w2 = np.random.rand(len(self.X[0][0]), output_size)* np.sqrt(1 / output_size)
        self.bias = np.random.rand()
        self.bias2 = np.random.rand()
        self.output_size = output_size
        #gonan try to do activation function to both 
        #hidden and final we can adjust this
        #self.activation = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
        #self.deriv = np.vectorize(lambda x: self.activation(x) * (1 - self.activation(x)))
        self.activation = expit
        self.deriv = lambda x: self.activation(x) * (1 - self.activation(x))
        self.count = 1
        
        
        for i in range(iterations):
            self.right_or_wrong = []
            for batch,Y in zip(self.X,self.y):
                batch_size = len(batch)
                batch_x = 0
                batch_hidden = 0
                errors = 0
                cost = 0
                for x,y in zip(batch,Y):
                    batch_x += x
                    hidden,final = self.forward(x.reshape(1,-1))
                    batch_hidden += hidden
                    temp_cost,temp_errors = self.back(hidden,final,y,x.reshape(1,-1))
                    errors += temp_errors
                    cost += temp_cost
                    self.right_or_wrong.append(1) if final.argmax() == y else self.right_or_wrong.append(0)
                self.update_weights(errors/batch_size,batch_hidden/batch_size,(batch_x/batch_size).reshape(1,-1))
            cost/= len(batch) 
            self.count += 1
            if self.count % 10 == 0:
                print(f'The cost after iteration {self.count}: {cost}')
                print(f'Correctly classified {sum(self.right_or_wrong)/len(self.right_or_wrong) * 100}%')
                print('------------------------------------------------------')
                    
    def update_weights(self,errors,hidden_output,x):
        layer_1_delta = (errors @ self.w2.T) * self.deriv(hidden_output)
        self.w2 -=  (hidden_output.T @ errors) * self.lr
        self.w1 -= (x.T @ layer_1_delta) * self.lr
    
    def batchify(self,X, y, batch_size):
        assert len(X) == len(y), "Input arrays must have the same length."
        assert batch_size > 0, "Batch size must be a positive integer."

        num_samples = len(X)
        num_batches = int(np.ceil(num_samples / batch_size))

        batched_X = [np.array(x) for x in np.array_split(X, num_batches)]
        batched_y = [np.array(y) for y in np.array_split(y, num_batches)]

        return np.array(batched_X), np.array(batched_y)
    
    def forward(self,X):
        hidden_output = self.activation(X @ self.w1 + self.bias)
        final_output = self.activation(hidden_output @ self.w2 + self.bias2)
        return hidden_output, final_output
        
    def back(self,hidden_output,final_outputs,y,x):
        
        y = np.array([0 if i!= y else 1 for i in range(self.output_size)])
        results = final_outputs - y
        cost = np.sum((final_outputs - y)**2) / len(results)
        
        layer_1_delta = results @ self.w2.T * self.deriv(hidden_output)
        
        
        #self.w2 -= self.lr * hidden_output.T @ results
        #self.w1 -= self.lr * x.T @ layer_1_delta
        return cost,results
    
    
    def testing(self):
        #self.testX=testX[:100]
        #self.testy=testy[:100]
        results = []
        for i,y in zip(self.testX,self.testy):
            one = self.activation(i.reshape(1,-1) @ self.w1 + self.bias)
            two = self.activation(one @ self.w2 + self.bias2)
            answer = two.argmax()
            results.append(1) if answer == y else results.append(0)
        print('TESTING RESULTS')
        print('----------------------------------')
        print(f'Percentage of correct classification: {sum(results)/len(results) * 100}')

In [44]:
#batchedNN = BatchNN(iterations=1000,batch_size=25, lr=.1, size=1000)

In [12]:
#batchedNN.testing()

# ADDING A FINITE DIFFERENCE TO CALCULATE BIAS UPDATE

1. CHANGE = HOW YOU ARE ALREADY HANDELING RESULTS IN UPDATES
2. ADD AN EPS = 0.01
3. PASS BACK EACH BIAS TO THE FORWARD PASS (USING THE AVERAGES OF THE INPUTS OF X ADD EPS TO BIAS
4. DO THIS FOR BOTH

In [13]:
from scipy.special import expit
class BatchNN:
    def __init__(self,X=X,y=train_labels,testX=y,testy=test_labels,
                 output_size=10,iterations=100,lr=0.1, batch_size=10, size=2500, eps=.1):
        
        np.random.seed(69)
        self.eps = eps
        self.lr = lr
        self.X,self.y = self.batchify(X[:size], y[:size],batch_size)
        #self.testX,self.testy = self.batchify(testX[:size], testy[:size],batch_size)
        self.testX = testX[:size]
        self.testy = testy[:size]
        self.w1 = np.random.rand(len(self.X[0][0]),len(self.X[0][0])) * np.sqrt(1 / len(self.X[0][0]))
        self.w2 = np.random.rand(len(self.X[0][0]), output_size)* np.sqrt(1 / output_size)
        self.bias = np.random.rand()
        self.bias2 = np.random.rand()
        self.output_size = output_size
        self.activation = expit
        self.deriv = lambda x: self.activation(x) * (1 - self.activation(x))
        self.count = 1
        
        
        for i in range(iterations):
            self.right_or_wrong = []
            for batch,Y in zip(self.X,self.y):
                batch_size = len(batch)
                batch_x = 0
                batch_hidden = 0
                errors = 0
                cost = 0
                for x,y in zip(batch,Y):
                    batch_x += x
                    hidden,final = self.forward(x.reshape(1,-1))
                    batch_hidden += hidden
                    temp_cost,temp_errors = self.back(hidden,final,y,x.reshape(1,-1))
                    errors += temp_errors
                    cost += temp_cost
                    self.right_or_wrong.append(1) if final.argmax() == y else self.right_or_wrong.append(0)
                self.update_weights(errors/batch_size,batch_hidden/batch_size,(batch_x/batch_size).reshape(1,-1))
                #NEW
                self.update_bias(errors/batch_size,batch_hidden/batch_size,(batch_x/batch_size).reshape(1,-1))
            cost/= len(batch) 
            self.count += 1
            if self.count % 10 == 0:
                print(f'The cost after iteration {self.count}: {cost}')
                print(f'Correctly classified {sum(self.right_or_wrong)/len(self.right_or_wrong) * 100}%')
                print('------------------------------------------------------')

   
    
    def update_bias(self, cost, hidden_output, x):
        true_f = self.activation(hidden_output @ self.w2 + self.bias2)

        bias_grad = np.mean(true_f) - np.mean(self.activation(hidden_output @ self.w2 + (self.bias2 + self.eps)))

        self.bias -= (bias_grad / self.eps) * self.lr
        self.bias2 -= (bias_grad / self.eps) * self.lr

        
                    
    def update_weights(self,errors,hidden_output,x):
        layer_1_delta = (errors @ self.w2.T) * self.deriv(hidden_output)
        self.w2 -=  (hidden_output.T @ errors) * self.lr
        self.w1 -= (x.T @ layer_1_delta) * self.lr
    
    def batchify(self,X, y, batch_size):
        assert len(X) == len(y), "Input arrays must have the same length."
        assert batch_size > 0, "Batch size must be a positive integer."

        num_samples = len(X)
        num_batches = int(np.ceil(num_samples / batch_size))

        batched_X = [np.array(x) for x in np.array_split(X, num_batches)]
        batched_y = [np.array(y) for y in np.array_split(y, num_batches)]

        return np.array(batched_X), np.array(batched_y)
    
    def forward(self,X):
        hidden_output = self.activation(X @ self.w1 + self.bias)
        final_output = self.activation(hidden_output @ self.w2 + self.bias2)
        return hidden_output, final_output
        
    def back(self,hidden_output,final_outputs,y,x):
        
        y = np.array([0 if i!= y else 1 for i in range(self.output_size)])
        results = final_outputs - y
        cost = np.sum((final_outputs - y)**2) / len(results)
        
        layer_1_delta = results @ self.w2.T * self.deriv(hidden_output)
        
        
        #self.w2 -= self.lr * hidden_output.T @ results
        #self.w1 -= self.lr * x.T @ layer_1_delta
        return cost,results
    
    
    def testing(self):
        #self.testX=testX[:100]
        #self.testy=testy[:100]
        results = []
        for i,y in zip(self.testX,self.testy):
            one = self.activation(i.reshape(1,-1) @ self.w1 + self.bias)
            two = self.activation(one @ self.w2 + self.bias2)
            answer = two.argmax()
            results.append(1) if answer == y else results.append(0)
        print('TESTING RESULTS')
        print('----------------------------------')
        print(f'Percentage of correct classification: {sum(results)/len(results) * 100}')

In [14]:
#batchedNN = BatchNN(iterations=1000,batch_size=25, lr=.1, size=1000)

In [15]:
#batchedNN.testing()

# Stochastic version With Bias update

In [16]:
#batchedNN = BatchNN(iterations=1000,batch_size=5, lr=.1, size=1000)

In [17]:
#batchedNN.testing()

In [21]:
#final is the output of last layer
#net_output_1 = layer_1
def back_prop(w1,w2,inputs,final,targets,net_output_1,lr=0.5):
    #IN REAL REVSION YOU'll just have adjust by be a 0
    # and then you'll just accumulate it and then divide by whatever
    # The batch size is
    updated_w1 = np.copy(w1)
    updated_w2 = np.copy(w2)
    deriv_error = np.vectorize(lambda x,y: -1*(y-x))
    deriv_sig_func = np.vectorize(lambda x: x * (1-x))
    
    #W2 update
    
    #GRADIENT OF ERROR
    #IN REAL VERSION GOING TO HAVE TO STACK IT
    #Don't know how many times I guess till it's the same shape 
    #AS W2
    deriv_error = np.vectorize(lambda x,y: -1*(y-x))
    error_derivs = deriv_error(final,targets)
    error_derivs = np.vstack(np.repeat(error_derivs,2,axis=0))
    
    #DERIV OF OUTPUT 
    sig_deriv = deriv_sig_func(final)
    sig_deriv = np.vstack(np.repeat(sig_deriv,2,axis=0))
    
    #DERIV OD PREVIOUS LAYER's raw output
    output_layer_1 = np.vstack(np.repeat(net_output_1, 2, axis=0))
    
    #said above gonna have two adjusted by vars for 1 and 2
    #if bathed we will acummuluate
    adjust_by = raw_output_1 * sig_deriv * error_derivs
    
    #Won't this won't be in batched version
    updated_w2 = (w2.T - adjust_by * lr).T
    
    # 
    
    

In [23]:
inputs = np.array([.05,.1]).reshape(1,-1)
w1 = np.array([.15,.2,.25,.3]).reshape(2,2)
w2 = np.array([.4,.45,.5,.55]).reshape(2,2)
outputs = np.array([.01,.99]).reshape(1,-1)
b1 = .35
b2 = .6


In [41]:
def forward(inputs=inputs,w1=w1,w2=w2,labels=outputs,b1=b1,b2=b2):
    sig = np.vectorize(lambda x: 1/ (1+ np.exp(-x)))
    
    #LAYER 1
    layer_1 = inputs @ w1.T + b1
    layer_1_output = sig(layer_1)
    print(f'raw output layer 1:{layer_1}')
    print(f'activated output layer 1: {layer_1_output}')
    
    #LAYER 2
    layer_2 = layer_1_output @ w2.T + b2
    layer_2_output = sig(layer_2)
    print(f'raw output layer 2:{layer_2}')
    print(f'activated output layer 2: {layer_2_output}')
    
    #ERROR
    erros_func = np.vectorize(lambda x,y: .5 *(y - x)**2)
    errors = erros_func(layer_2_output, labels)
    total_error = errors.sum()
    print(f'individual errors: {errors}')
    print(f'Total error: {total_error}')